## Agents: extra section HW

### Preparation
First, we'll define a function that we will use when building our agent.

It will generate fake weather data:

In [ ]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

### Q1. Define function description
We want to use it as a tool for our agent, so we need to describe it

How should the description for this function look like? Fill in missing parts

In [12]:
TODO1 = "weather-tool"
TODO2 = "This tool gets the weather at a specified location"
TODO3 = "query"
TODO4 = "For a specific location, and unless otherwise stated, the current date and time, look up the weather"
TODO5 = TODO3

get_weather_tool = {
    "type": "function",
    "name": TODO1,
    "description": TODO2,
    "parameters": {
        "type": "object",
        "properties": {
            TODO3: {
                "type": "string",
                "description": TODO4
            }
        },
        "required": [TODO5],
        "additionalProperties": False
    }
}

What did you put in TODO3?

### A1: See below

In [ ]:
import json
print(json.dumps(get_weather_tool,indent=2))

{
  "type": "function",
  "name": "weather-tool",
  "description": "This tool gets the weather at a specified location",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {
        "type": "string",
        "description": "For a specific location, and unless otherwise stated, the current date and time, look up the weather"
      }
    },
    "required": [
      "query"
    ],
    "additionalProperties": false
  }
}


### Q2. Adding another tool

Let's add another tool - a function that can add weather data to our database:

In [ ]:
def set_weather(city: str, temp: float) -> None:
    """
    Stores or updates the temperature for a given city in the known_weather_data dictionary.

    Args:
        city (str): Name of the city (case-insensitive, leading/trailing spaces will be removed).
        temp (float): Temperature value (in degrees, assumed to be Celsius or Fahrenheit depending on context).

    Returns:
        str: Confirmation string "OK" indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return "OK"


Now let's write a description for it.

What did you write?

## A2. We wrote the following:
```bash

"""
    Stores or updates the temperature for a given city in the known_weather_data dictionary.

    Args:
        city (str): Name of the city (case-insensitive, leading/trailing spaces will be removed).
        temp (float): Temperature value (in degrees, assumed to be Celsius or Fahrenheit depending on context).

    Returns:
        str: Confirmation string "OK" indicating successful update.
    """
```

Optionally, you can test it after adding this function.

### MCP
MCP stands for Model-Context Protocol. It allows LLMs communicate with different tools (like Qdrant). It's function calling, but one step further:

A tool can export a list of functions it has
When we include the tool to our Agent, we just need to include the link to the MCP server


### Q3. Install FastMCP
Let's install a library for MCP - FastMCP:

```bash
pip install fastmcp
```

What's the version of FastMCP you installed?

### A3. `2.10.5`

In [1]:
import fastmcp
print(fastmcp.__version__)


2.10.5


### Q4. Simple MCP Server
A simple MCP server from the documentation looks like that:

```bash
# weather_server.py
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

@mcp.tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

if __name__ == "__main__":
    mcp.run()
```

In our case, we need to write docstrings for our functions.

Let's ask ChatGPT for help:

In [ ]:
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

Let's change the example for our case and run it

What do you see in the output?

Look for a string that matches this template:

```bash
Starting MCP server 'Demo 🚀' with transport '<TODO>'
```

### A4: We edited the base code and saved it onto weather_server.py. After running:

```bash
python ./homeworks/weather_server.py
```

We get: Starting MCP server 'Demo 🚀' with transport `'stdio'`

### Q5. Protocol
There are different ways to communicate with an MCP server. Ours is currently running using standart input/output, which means that the client write something to stdin and read the answer using stdout.

Our weather server is currently running.

This is how we start communitcating with it:

- First, we send an initialization request -- this way, we register our client with the server:

```bash
{"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}
```

- We should get back something like that, which is an aknowledgement of the request:

```bash
{"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":true}},"serverInfo":{"name":"Demo 🚀","version":"1.9.4"}}}
```

- Next, we reply back, confirming the initialization:
```bash
{"jsonrpc": "2.0", "method": "notifications/initialized"}
```

We don't expect to get anything in response
- Now we can ask for a list of available methods:
```bash
{"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
```

- Let's ask the temperature in Berlin:
```bash
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city": "Berlin"}}}
```

### A5. We get a random number, like 21.1C

### Q6. Client
We typically don't interact with the server by copy-pasting commands in the terminal.

In practice, we use an MCP Client. Let's implement it.

FastMCP also supports MCP clients:

```bash
from fastmcp import Client

async def main():
    async with Client(<TODO>) as mcp_client:
        # TODO
```

Use the client to get the list of available tools of our script. How does the result look like?

If you're running this code in Jupyter, you need to pass an instance of MCP server to the Client:

```bash
import weather_server

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        # ....
```bash

If you run it in a script, you will need to use asyncio:

```bash
import asyncio

async def main():
    async with Client("weather_server.py") as mcp_client:
        # ...

if __name__ == "__main__":
    test = asyncio.run(main())

```

Copy the output with the available tools when filling in the homework form.

In [9]:
import nest_asyncio
import asyncio
from fastmcp import Client

nest_asyncio.apply()

async def run_mcp_client():
    async with Client("http://localhost:8765/mcp") as mcp_client:
        # get the list of tools
        tools = await mcp_client.list_tools()
        print("Available tools:")
        print(tools)
        # for tool in tools:
        #     print(f"- {tool['name']}: {tool.get('description', '')}")
        
        # # Set the weather
        # set_result = await mcp_client.call_tool("set_weather", {"city": "Berlin", "temp": 22.5})
        # print("Set result:", set_result)

        # # Get the weather
        # get_result = await mcp_client.call_tool("get_weather", {"city": "Berlin"})
        # print("Weather in Berlin:", get_result)

await run_mcp_client()


Available tools:
[Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'tem